In [1]:
import requests
import xml.etree.cElementTree as et
import json
import pandas as pd
import numpy as np

In [3]:
from tqdm import tqdm_notebook as tqdmn

In [9]:
from io import StringIO

In [131]:
# ids = [5219, 5247, 5248]
ids = [4981, 4982, 4983, 4984, 4985]

In [113]:
flags_url = 'http://rating.chgk.info/tournaments.php?tournament_id={}&download_data=tournament_export_teams_with_flags'

In [132]:
results_pd = [pd.read_csv(StringIO(requests.get(flags_url.format(i)).text), skiprows=1, index_col=None,
                          sep=';', error_bad_lines=False, 
                          names=['idteam','name_{}'.format(i), 'city_{}'.format(i),'place_{}'.format(i),'questions_{}'.format(i),'flags_{}'.format(i)]) for i in tqdmn(ids)]

A Jupyter Widget

In [ ]:
results_json = [requests.get('http://rating.chgk.info/api/tournaments/{}/list'.format(id)).text for id in tqdmn(ids)]
results_pd = [pd.read_json(res) for res in results_json]

In [133]:
results_pd[0].head()

,idteam,name_4981,city_4981,place_4981,questions_4981,flags_4981
0,27177,Призраки Коши,Санкт-Петербург,"1,5",31,NaN
1,42167,Шесть пик,Санкт-Петербург,"1,5",31,NaN
2,209,Команда Коваленко,Великий Новгород,5,30,NaN
3,928,ОМ,Воронеж,5,30,NaN
4,46440,Утиный макгаффин,Киев,5,30,NaN


In [116]:
from functools import reduce

In [217]:
merged_df =  reduce(lambda l, r: pd.merge(l, r, on=['idteam'], how='outer'), results_pd)

In [218]:
merged_df.head()

,idteam,name_4981,city_4981,place_4981,questions_4981,flags_4981,name_4982,city_4982,place_4982,questions_4982,...,name_4984,city_4984,place_4984,questions_4984,flags_4984,name_4985,city_4985,place_4985,questions_4985,flags_4985
0,27177,Призраки Коши,Санкт-Петербург,"1,5",31.0,NaN,Призраки Коши,Санкт-Петербург,5,25.0,...,Призраки Коши,Санкт-Петербург,"2,5",33.0,NaN,Призраки Коши,Санкт-Петербург,"3,5",28.0,NaN
1,42167,Шесть пик,Санкт-Петербург,"1,5",31.0,NaN,Candy Famine,Санкт-Петербург,"481,5",13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,209,Команда Коваленко,Великий Новгород,5,30.0,NaN,Команда Коваленко,Великий Новгород,1,28.0,...,Команда Коваленко,Великий Новгород,52,27.0,NaN,Команда Коваленко,Великий Новгород,"7,5",27.0,NaN
3,928,ОМ,Воронеж,5,30.0,NaN,ОМ,Воронеж,"36,5",22.0,...,ОМ,Воронеж,131,24.0,NaN,ОМ,Воронеж,119,19.0,NaN
4,46440,Утиный макгаффин,Киев,5,30.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [219]:
len(merged_df)

1784

In [220]:
merged_df['name'] = [' / '.join(set(r.dropna().tolist())) for i, r in merged_df.loc[:,merged_df.columns.str.startswith('name_')].iterrows()]
merged_df = merged_df.drop(merged_df.columns[merged_df.columns.str.startswith('name_')], axis=1)
merged_df.head()

,idteam,city_4981,place_4981,questions_4981,flags_4981,city_4982,place_4982,questions_4982,flags_4982,city_4983,...,flags_4983,city_4984,place_4984,questions_4984,flags_4984,city_4985,place_4985,questions_4985,flags_4985,name
0,27177,Санкт-Петербург,"1,5",31.0,NaN,Санкт-Петербург,5,25.0,NaN,Санкт-Петербург,...,NaN,Санкт-Петербург,"2,5",33.0,NaN,Санкт-Петербург,"3,5",28.0,NaN,Призраки Коши
1,42167,Санкт-Петербург,"1,5",31.0,NaN,Санкт-Петербург,"481,5",13.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Candy Famine / Шесть пик
2,209,Великий Новгород,5,30.0,NaN,Великий Новгород,1,28.0,NaN,Великий Новгород,...,NaN,Великий Новгород,52,27.0,NaN,Великий Новгород,"7,5",27.0,NaN,Команда Коваленко
3,928,Воронеж,5,30.0,NaN,Воронеж,"36,5",22.0,NaN,Воронеж,...,NaN,Воронеж,131,24.0,NaN,Воронеж,119,19.0,NaN,ОМ
4,46440,Киев,5,30.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Утиный макгаффин


Chech whether all flags and cities are equal, merge them

In [221]:
for param in ['flags', 'city']:
    param_cols = merged_df.loc[:,merged_df.columns.str.startswith(param+'_')]
    param_join = param_cols.apply(lambda x: set([e for c in x.dropna() if not pd.isna(c) for e in set(c.split())]), axis=1)
    same = param_join.apply(lambda x: len(x) < 2)
    param_join = param_join.apply(lambda x: ' / '.join(x))
    
    if not np.all(same):
        diff_ids = merged_df.loc[np.logical_not(same), merged_df.columns.str.contains('(idteam)|(name)|({}_)'.format(param))]
#         print('These teams had different {}:'.format(param))
#         print(diff_ids)
        
    merged_df = merged_df.drop(merged_df.columns[merged_df.columns.str.startswith(param)], axis=1)
    merged_df[param] = param_join

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [222]:
merged_df['questions'] = np.nansum(merged_df.loc[:,merged_df.columns.str.startswith('questions_')], axis=1)
merged_df['place'] = np.nansum(np.array(merged_df.loc[:,merged_df.columns.str.startswith('place_')].apply(lambda x: x.str.replace(',','.')), dtype=np.float), axis=1)
merged_df.sort_values(['questions','place'], inplace=True, ascending=[False,True])
merged_df.replace(np.nan, 0, inplace=True)

Remove worst tour

In [223]:
np.nanmin(merged_df.loc[:,merged_df.columns.str.startswith('questions_')], axis=1)

array([25., 27., 23., ...,  0.,  0.,  0.])

In [224]:
merged_df['wo_worst_questions'] = merged_df.questions - np.nanmin(merged_df.loc[:,merged_df.columns.str.startswith('questions_')], axis=1)
merged_df.sort_values(['wo_worst_questions','questions'], inplace=True, ascending=[False,False])
merged_df = merged_df[merged_df.columns.tolist()[:1] + merged_df.columns.tolist()[-6:] + merged_df.columns.tolist()[1:-6]]

In [225]:
merged_df.columns.tolist()[:1] + merged_df.columns.tolist()[:1]

['idteam', 'idteam']

In [226]:
merged_df.head(20)

,idteam,name,flags,city,questions,place,wo_worst_questions,place_4981,questions_4981,place_4982,questions_4982,place_4983,questions_4983,place_4984,questions_4984,place_4985,questions_4985
0,27177,Призраки Коши,,Санкт-Петербург,143.0,30.0,118.0,"1,5",31.0,5,25.0,"17,5",26.0,"2,5",33.0,"3,5",28.0
2,209,Команда Коваленко,,Новгород / Великий,139.0,76.5,112.0,5,30.0,1,28.0,11,27.0,52,27.0,"7,5",27.0
54,6936,Хронически разумные United,,Минск,132.0,116.0,109.0,66,25.0,"17,5",23.0,11,27.0,9,31.0,"12,5",26.0
72,55517,Das Gemeine,С,Санкт-Петербург,124.0,455.5,109.0,66,25.0,348,15.0,4,29.0,25,29.0,"12,5",26.0
11,38722,Хайвмайнд,,Томск,130.0,131.0,108.0,11,29.0,"36,5",22.0,11,27.0,"16,5",30.0,56,22.0
158,4622,Спонсора.net,,Воронеж,129.0,245.5,107.0,170,22.0,"17,5",23.0,52,23.0,5,32.0,1,29.0
3,928,ОМ,,Воронеж,126.0,292.5,107.0,5,30.0,"36,5",22.0,1,31.0,131,24.0,119,19.0
106,58596,Взрослая неожиданность / Грейпфрут на грядке /...,С,сборная / Москва,126.0,210.0,105.0,98,24.0,"59,5",21.0,"25,5",25.0,9,31.0,18,25.0
19,45174,София,,Саранск,125.0,182.0,104.0,"18,5",28.0,"59,5",21.0,"25,5",25.0,52,27.0,"26,5",24.0
84,3730,Фаг лямбда,,Санкт-Петербург,125.0,245.5,104.0,98,24.0,5,25.0,87,21.0,52,27.0,"3,5",28.0


In [210]:
merged_df.to_csv('ovsch_merged.csv', index=False)